In [1]:
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from datetime import datetime

In [2]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [3]:
from dqn import DQN
from environment import LunarLanderEnvManager

In [4]:
# EXPERIENCE CLASS

Experience = namedtuple('Experience', ['state', 'action', 'next_state', 'reward', 'done'])

In [5]:
# REPLAY MEMORY CLASS

class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.next_location = 0
        self.per = PrioritizedExperienceReplay(capacity)

    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.next_location] = experience
        self.per.push(self.next_location)
        self.next_location = (self.next_location+1) % self.capacity
        
    def prioritized_sample(self, batch_size):
        indices = self.per.sample_indices(batch_size)
        sample = []
        for i in indices:
            sample.append(self.memory[i])
        importance = torch.tensor(self.per.generate_importance())
        return sample, importance

    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

In [6]:
class PrioritizedExperienceReplay():
    def __init__(self, capacity):
        self.capacity = capacity
        self.priorities = []
        self.indices = []
        self.sum = 0
        self.max = 1
        
        self.beta = 0
        self.beta_step = 0.0001
        self.beta_end = 1
        
    def push(self, next_location):
        if len(self.priorities) < self.capacity:
            self.priorities.append(self.max)
            self.sum += self.max
        else:
            self.sum -= self.priorities[next_location]
            self.priorities[next_location] = self.max
            self.sum += self.priorities[next_location]
        
    def sample_indices(self, batch_size):
        self.indices = random.choices(range(len(self.priorities)), k=batch_size, \
                                      weights=self.priorities)
        self.update_max()
        return self.indices
    
    def update_max(self):
        self.max = 1
        for i in self.indices:
            if self.priorities[i] > self.max:
                self.max = self.priorities[i]
    
    def generate_importance(self):
        importance = []
        for i in self.indices:
            importance.append(((1 / len(self.priorities)) * (self.sum / self.priorities[i])) ** self.beta)
        importance_max = max(importance)
        importance_normalized = [i / importance_max for i in importance]
        if self.beta < self.beta_end:
            self.beta += self.beta_step
        return importance_normalized
            
    def update_priorities(self, criterion, offset=0.01):
        for i, j in zip(self.indices, range(len(criterion))):
            self.sum -= self.priorities[i]
            self.priorities[i] = criterion[j] + offset
            self.sum += self.priorities[i]

In [7]:
# EPSILON GREEDY STRATEGY CLASS

class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
        
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * math.exp(-1. * current_step * self.decay)

In [8]:
# AGENT CLASS

class Agent():
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device

    def select_action(self, state, policy_net):
        rate = self.strategy.get_exploration_rate(self.current_step)
        self.current_step += 1
        if rate > random.random():
            action = random.randrange(self.num_actions)
            return torch.tensor([action]).to(self.device)
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).to(self.device)

In [9]:
# TRAINING PERFORMANCE PLOTS

def plot(rewards_per_episode, moving_average, losses):
    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    plt.title('Reward')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.plot(rewards_per_episode)
    plt.plot(moving_average)

    plt.subplot(1,2,2)
    plt.title('Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.plot(losses)
    plt.pause(0.001)
    
def get_moving_average(period, values):
    values = torch.tensor(values, dtype=torch.float)
    if len(values) >= period:
        moving_avg = values.unfold(dimension=0, size=period, step=1).mean(dim=1).flatten(start_dim=0)
        moving_avg = torch.cat((torch.zeros(period-1), moving_avg))
        return moving_avg.numpy()
    else:
        moving_avg = torch.zeros(len(values))
        return moving_avg.numpy()

In [10]:
# TENSOR PROCESSING FUNCTION

def extract_tensors(expriences):
    batch = Experience(*zip(*experiences))
    t1 = torch.cat(batch.state)
    t2 = torch.cat(batch.action)
    t3 = torch.cat(batch.reward)
    t4 = torch.cat(batch.next_state)
    t5 = torch.cat(batch.done)
    return (t1, t2, t3, t4, t5)

In [11]:
# Q-VALUE CALCULATOR CLASS

class QValues():
    
    @staticmethod
    def get_current(policy_net, states, actions):
        return policy_net(states).gather(dim=1, index=actions.unsqueeze(-1))
    
    @staticmethod
    def get_next(target_net, next_states):
        return target_net(next_states).max(dim=1)[0]

In [12]:
# MAIN PROGRAM

# SETTINGS
num_episodes = 1000
render = False

# HYPERPARAMETERS

batch_size = 32
gamma = 0.99
eps_start = 1
eps_end = 0.01
eps_decay = 0.001
alpha = 0.3
target_update = 500
memory_size = 100000
lr = 0.001

# OBJECT INSTANCES

device = torch.device("cuda")
em = LunarLanderEnvManager(device)
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, em.num_actions_available(), device)
memory = ReplayMemory(memory_size)

policy_net = DQN(em.num_states_available()).to(device)
target_net = DQN(em.num_states_available()).to(device)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)

rewards_per_episode = []
losses = []

# MAIN LOOP

for episode in range(num_episodes):
    em.reset()
    state = em.get_state()
    total_reward = 0
    
    # timestep loop
    for timestep in count():
        action = agent.select_action(state, policy_net)
        reward = em.take_action(action)
        done = em.if_done()
        next_state = em.get_state()        
        memory.push(Experience(state, action, next_state, reward, done))
        state = next_state
        total_reward += reward.item()
        
        if memory.can_provide_sample(batch_size):
            experiences, importance = memory.prioritized_sample(batch_size)
            states, actions, rewards, next_states, dones = extract_tensors(experiences)

            current_q_values = QValues.get_current(policy_net, states, actions)
            current_q_values = current_q_values.float()

            next_q_values = QValues.get_next(target_net, next_states)
            next_q_values = next_q_values.float()

            target_q_values = (next_q_values * gamma) * (1 - dones) + rewards
            target_q_values = target_q_values.float()

            td_error = current_q_values - target_q_values.unsqueeze(1)
            loss = torch.mean(torch.square(torch.multiply(td_error, importance.to(device))))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            memory.per.update_priorities(td_error.detach().flatten().abs().pow(alpha).cpu().numpy())
            losses.append(loss.item())
            
            if agent.current_step % target_update == 0:
                target_net.load_state_dict(policy_net.state_dict())
                
            if render:
                em.render()
            
        if em.done:
            rewards_per_episode.append(total_reward)
            moving_average = get_moving_average(50, rewards_per_episode)
            plot(rewards_per_episode, moving_average, losses)
            print("Episode", len(rewards_per_episode))
            print("Average", moving_average[-1])
            print("Steps", timestep)
            if is_ipython: display.clear_output(wait=True)
            break
    
    if moving_average[-1] >= 200:
        date_time = datetime.now().strftime("%y%m%d_%H%M%S")
        FILE = "model_" + date_time + ".pth"
        torch.save(policy_net.state_dict(), FILE)
        break
        
em.close()

KeyboardInterrupt: 